## COMP6713  - Natural Language Processing – 25T1

### Tutorial – Week 5


Following the in-class demo, you have a notebook that demonstrates parameter-efficient fine-tuning (PEFT) of a decoder model for causal text generation. Specifically, the notebook uses LoRA to fine-tune OPT using an unlabeled dataset of quotes. The notebook also shows how the fine-tuned model can be used to complete sentences provided as prompts. For example, the prompt may be an incomplete sentence such as “People expect “ where the output completes the sentence.

In this tutorial, you will extend the notebook to use a labelled dataset to fine-tune a model. In other words, your language model learns to complete a prompt which contains the text to be classified, along with the label to be predicted. In doing so, it is learning a classification task via a decoder model! Therefore, you will receive a labelled dataset and structure its instances in the form of a prompt. The fine-tuned model will use a prompt such as: “What is the sentiment of this sentence?\n Sentence: I love the movie.\n Label: “

The completion of this tutorial will enable you with skills in (a) zero-shot prompting, and (b) prompt tuning.

This tutorial is based on the PEFT library from HuggingFace: https://github.com/huggingface/peft/


In [ ]:
!pip install peft datasets

In [ ]:
# Source: https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu" # What does this line do?
print(device)

# Initialise the model and tokenizer to a pre-trained model. Suggestions: facebook/opt-350m, bigscience/bloom-560m
model = # TODO
tokenizer = # TODO

**Part 1: Use the LM to complete a few prompts**

In [ ]:
prompt = "Australia announced "
# Encode the prompt using the tokenizer. If using BLOOM, use this: https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/bloom#transformers.BloomTokenizerFast

# Else, use appropriate documentation
inputs = # TODO

# Complete the prompt with a maximum length of 50. 
# What happens if you set the maximum length to 5? Why does that happen?
# Hint: Use model.generate
outputs = # TODO
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# printing output
print(text)

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)
# Understand the code above:
# What are the other TaskTypes in PEFT? When would you use them?
# Answer:
# What does r mean?
# Answer:

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# What does the output mean?
# Answer:

In [ ]:
type(model)

In [ ]:
import transformers
from datasets import load_dataset
# Choose a dataset. Select the right column in the dataset.
data = # TODO 
column = #TODO
data = data.map(lambda samples: tokenizer(samples[column]), batched=True)
#TODO: Select a small sample of the dataset if your dataset is too big.
data = #TODO

In [ ]:
import transformers
from datasets import load_dataset

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        # TODO Add arguments corresponding to number of steps, learning rate, gradient accumulation step and batch size
        output_dir="./outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

In [ ]:
# Set the model in evaluation mode
#TODO 

input = tokenizer("Australia announced ", return_tensors="pt").to(device)

with torch.no_grad():
    output_tokens = model.generate(**input, do_sample=False, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, max_new_tokens=200)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
# Extra: Does the output get any better if:
# you increase the number of data samples?
# you increase the number of steps?